In [1]:
import os
from dotenv import find_dotenv, load_dotenv
from openai import OpenAI
import openai
import psycopg2

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Set up your OpenAI API key
# openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
print(OPENAI_API_KEY[:12])

sk-proj-Mi2f


In [4]:
print('Connecting to PostgreSQL...')
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="host.docker.internal",
)

cursor = conn.cursor()

print('Successfully connected to PostgreSQL.')

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [6]:
# Open the SQL file
with open("./movie_lite1.sql", "r") as f:
    sql = f.read()

# Split the SQL file into individual statements
statements = sql.split(";")

# Create a cursor object
cursor = conn.cursor()

# Execute each statement
for statement in statements:
    statement = statement.strip()
    if statement:  # Ignore empty statements
        cursor.execute(statement)

# Commit the changes
conn.commit()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 41002: character maps to <undefined>

In [5]:
# Open the SQL file
with open("./movie_lite1.sql", "r") as f:
    sql = f.read()

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute(sql)

# Commit the changes
conn.commit()



UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 41002: character maps to <undefined>

Finally, load the dataset into Postgres:

## Perform Vector Similarity Search

The dataset already stores a vectorized representation of movies' overviews in the `overview_vector` column. Each vector is a 1536-dimensional embedding generated with the OpenAI `text-embedding-ada-002` model.

First, define several functions that generate vectors for user prompts and configure the matching threshold and count parameters for the similarity search:

In [6]:
# Converting the prompt to the pgvector embedding
def get_embedding(prompt):
    response = openai.embeddings.create(
        input=prompt,
        model='text-embedding-ada-002')

    embedding = response.data[0].embedding

    # Converting the embedding to the pgvector and returning it
    return '[' + ','.join(map(str, embedding)) + ']'

# Getting the matching threshold for the similarity search
def get_matching_threshold():
    return 0.7

# Getting the number of matching movies to return
def get_matching_count():
    return 3

Second, find the most relevant movies for a provided user prompt by calculating the cosine distance (`<=>`) between the prompt's and movies' embeddings:

In [16]:
user_prompt = 'A movie about an adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'SELECT title, overview '
    'FROM movie WHERE 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

('A Brief Season', 'Film directed by Renato Castellani.')
('A Brief Season', 'Film directed by Renato Castellani.')
("Franz Kafka's a Country Doctor", 'A hapless country doctor describes with breathless urgency a night-time summons to attend a young patient. Events soon take on a surreal aspect as "unearthly horses" transport him instantaneously to the bedside. The doctor, preoccupied with personal distractions and grievances against those he is employed to care for, fails to find what is revealed to be a vile, fatal wound. He is humiliated by the villagers, who are "always expecting the impossible from the doctor," and doomed to an endless return trip, losing everything.')


## Pre-Filter Data Before Similarity Search

As a general-purpose relational database, PostgreSQL allows you to pre-filter data before a vector search is started. You can pre-filter by specifying a condition on non-vector columns in the `WHERE` clause of a query statement.

For instance, imagine the user selecting the `Science Fiction` category and asking to suggest movies with a rating of `7` or higher. Then, the user prompts for `A movie about a space adventure`. The final SQL query can look as follows:

In [14]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'SELECT title, vote_average, genres '
    'FROM movie WHERE vote_average >= 7 '
    'AND genres @> \'[{"name": "Science Fiction"}]\' '
    'AND 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

PostgreSQL will pre-filter data using the following columns:

* The `vote_average` column stores a rank from `1` through `10`. 
* The `genres` column is an array of JSON objects stored in the JSONB format. A movie can be categorized by several genres, with a sample value looking as follows - `[{'id': 12, 'name': 'Adventure'}, {'id': 18, 'name': 'Drama'}, {'id': 878, 'name': 'Science Fiction'}])`

Run the `EXPLAIN` statement if you'd like to see the actual execution plan:

In [9]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'EXPLAIN (costs off) SELECT title, vote_average, genres '
    'FROM movie WHERE vote_average >= 7 '
    'AND genres @> \'[{"name": "Science Fiction"}]\' '
    'AND 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

('Limit',)
('  ->  Sort',)
("        Sort Key: ((overview_vector <=> '[0.015956813,-0.038707,-0.028187415,-0.033738714,-0.004005046,0.00923949,-0.0054118824,-0.03206572,0.0041444623,-0.00048756052,0.030519467,-0.0014147572,0.015297756,-0.017984686,0.021356022,-0.009188793,0.031482704,-0.011210327,0.007008831,-0.0005497433,0.0057540857,-0.016020184,0.012870647,0.013662784,0.0075094616,-0.015082295,0.026438376,-0.013383952,0.026590466,-0.0138655715,0.025361069,-0.0060772775,0.018960599,-0.008041779,-0.01730028,-0.0070468537,0.0017553763,-0.015348453,0.013054423,-0.018491654,0.010316797,0.02539909,-0.022293912,-0.0089669945,-0.017769225,-0.007769283,0.02472736,-0.030848997,-0.012718556,-0.0033206397,0.0077819573,0.02841555,-0.038149334,-0.014397888,0.004838375,-0.0066666277,-0.020557547,0.0071165618,0.017693179,-0.019492915,-0.0031020097,0.0032255831,-0.006143817,0.012826287,-0.014245798,0.015906118,-0.0028184245,-0.011413114,0.021064516,-0.010285111,0.02762975,0.017490393,0.0038656301,0.

The generated plan should look as follows:
```sql
('Limit',)
('  ->  Sort',)
("        Sort Key: ((overview_vector <=> '[0.015902195,-0.03861236,-0.02809557,...]'::vector))",)
('        ->  Seq Scan on movie',)
('              Filter: ((vote_average >= \'7\'::numeric) AND (genres @> \'[{"name": "Science Fiction"}]\'::jsonb) AND ((\'1\'::double precision - (overview_vector <=> \'[0.015902195,-0.03861236,-0.02809557,...]\'::vector)) >= \'0.7\'::double precision))',)

```

The plan indicates that the data is initially filtered by the `vote_average` and `genres` columns, followed by the similarity search on the `overview_vector` column. Note that PostgreSQL may use a different plan if you create an index for any of the columns or if other conditions change.

## Optimize Vector Search With HNSW Index

Currently, the embeddings stored in the `overview_vector` column are not indexed. This means that the database performs an exact nearest neighbor search by comparing a user prompt's vector to all the movies' overview embeddings. You can confirm this by checking the execution plan, which will show the `Seq Scan` (full table scan) access method on the `movie` table:

In [13]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'EXPLAIN (costs off) SELECT title, overview '
    'FROM movie WHERE 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

('Limit',)
('  ->  Sort',)
("        Sort Key: ((overview_vector <=> '[0.015956813,-0.038707,-0.028187415,-0.033738714,-0.004005046,0.00923949,-0.0054118824,-0.03206572,0.0041444623,-0.00048756052,0.030519467,-0.0014147572,0.015297756,-0.017984686,0.021356022,-0.009188793,0.031482704,-0.011210327,0.007008831,-0.0005497433,0.0057540857,-0.016020184,0.012870647,0.013662784,0.0075094616,-0.015082295,0.026438376,-0.013383952,0.026590466,-0.0138655715,0.025361069,-0.0060772775,0.018960599,-0.008041779,-0.01730028,-0.0070468537,0.0017553763,-0.015348453,0.013054423,-0.018491654,0.010316797,0.02539909,-0.022293912,-0.0089669945,-0.017769225,-0.007769283,0.02472736,-0.030848997,-0.012718556,-0.0033206397,0.0077819573,0.02841555,-0.038149334,-0.014397888,0.004838375,-0.0066666277,-0.020557547,0.0071165618,0.017693179,-0.019492915,-0.0031020097,0.0032255831,-0.006143817,0.012826287,-0.014245798,0.015906118,-0.0028184245,-0.011413114,0.021064516,-0.010285111,0.02762975,0.017490393,0.0038656301,0.

Postgres pgvector supports the IVFFlat and HNSW indexes, which are two of the most widespread index types across vector databases.

Let's create an HNSW index on the embeddings stored in the `overview_vector` column:

In [11]:
print('Building the index. It might take a minute...')

# Build the index
cursor.execute(
    'CREATE INDEX movie_overview_hnsw_idx ON movie '
    'USING hnsw (overview_vector vector_cosine_ops) '
    'WITH (m = 4, ef_construction = 10)')
conn.commit()

# Update the statistics for the query planner
# to ensure that the index is used for the vector similarity search
conn.autocommit = True
cursor.execute(
    'VACUUM ANALYZE movie')
conn.autocommit = False

print('HNSW Index created.')

Building the index. It might take a minute...
HNSW Index created.


Check the execution plan once more for the previous query to ensure that Postgres now expedites the similarity search with the newly created index. You will see the `Index Scan` access method on the `movie` table:

In [12]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'EXPLAIN (costs off) SELECT title, overview '
    'FROM movie WHERE 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

('Limit',)
('  ->  Sort',)
("        Sort Key: ((overview_vector <=> '[0.015956813,-0.038707,-0.028187415,-0.033738714,-0.004005046,0.00923949,-0.0054118824,-0.03206572,0.0041444623,-0.00048756052,0.030519467,-0.0014147572,0.015297756,-0.017984686,0.021356022,-0.009188793,0.031482704,-0.011210327,0.007008831,-0.0005497433,0.0057540857,-0.016020184,0.012870647,0.013662784,0.0075094616,-0.015082295,0.026438376,-0.013383952,0.026590466,-0.0138655715,0.025361069,-0.0060772775,0.018960599,-0.008041779,-0.01730028,-0.0070468537,0.0017553763,-0.015348453,0.013054423,-0.018491654,0.010316797,0.02539909,-0.022293912,-0.0089669945,-0.017769225,-0.007769283,0.02472736,-0.030848997,-0.012718556,-0.0033206397,0.0077819573,0.02841555,-0.038149334,-0.014397888,0.004838375,-0.0066666277,-0.020557547,0.0071165618,0.017693179,-0.019492915,-0.0031020097,0.0032255831,-0.006143817,0.012826287,-0.014245798,0.015906118,-0.0028184245,-0.011413114,0.021064516,-0.010285111,0.02762975,0.017490393,0.0038656301,0.